In [1]:
import numpy as np
import pandas as pd
import pickle


In [ ]:
import json
import re
import sys
import time
import warnings
from random import sample

import openai
from openai import AzureOpenAI
import pandas as pd
# from tqdm import tqdm
import tiktoken
key = "insert_key_here" # this key does not work anymore

# Parameters
client = AzureOpenAI(
  azure_endpoint = "https://hkust.azure-api.net",
  api_version = "2024-06-01",
  api_key = key #put your api key here
)

# Function
def get_response(message, instruction):
    response = client.chat.completions.create(
		model = 'gpt-4',
        temperature = 1,
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": message}
        ]
    )
    
    # print token usage
    print(response.usage)
    # return the response
    return response.choices[0].message.content
  
def LM_generation(prompt, max_tokens = 10, temperature = 0.1, model = "gpt-4", system_msg = None, msg_history = None):
    retry_attempt = 5
    retry_num = 0
    generation_success = False
    messages = []
    if system_msg is not None:
        messages.append({'role':'system', 'content': system_msg})
    if msg_history is not None:
        messages = msg_history
    messages.append({'role':'user', 'content':prompt})
    while retry_num < retry_attempt and not generation_success:
        try:
            gen = client.chat.completions.create(
                model = model,
                messages = messages,
                max_tokens = max_tokens,
                temperature = temperature,
            ) 
            generation_success = True
            input_tokens = gen.usage.prompt_tokens
            output_tokens = gen.usage.completion_tokens
        except openai.APIError as e:
            retry_num += 1
            generation_success = False
            time.sleep(5)
            # return "openai.error.APIError", -1, -1 
        except openai.RateLimitError as e:
            retry_num += 1
            generation_success = False
            time.sleep(30)
        except:
            retry_num += 1
            generation_success = False
            time.sleep(10)
    
    if generation_success:
        if gen == None or gen.choices[0] == None or gen.choices[0].message == None or gen.choices[0].message.content == None:
            return "", 0, 0
        return gen.choices[0].message.content.strip(), input_tokens, output_tokens
    else:
        return "Error", -1,-1

def count_tokens(text, encoding_name = "cl100k_base"):
    encoding = tiktoken.get_encoding(encoding_name)
    # print(text)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [ ]:
import json
import numpy as np

# Open and read the JSON file
with open('llm_ready_opentom_vfin.jsonl') as f: # ambiguous story
    final_opentom = [json.loads(line) for line in f]
    
with open('llm_ready_bigtom_vfin.jsonl') as f: # ambiguous story
    final_bigtom = [json.loads(line) for line in f]

# add filter:
bigtom_filter = np.load('bigtom_filter.npy', allow_pickle=True)
opentom_filter = np.load('opentom_filter.npy', allow_pickle=True)

final_opentom = np.array(final_opentom)[opentom_filter]
final_bigtom = np.array(final_bigtom)[bigtom_filter]

In [6]:
#save

def save_list_of_dict(save_name, list_of_dict):
    with open(save_name, 'wb') as f:
        pickle.dump(list_of_dict, f)

def load_list_of_dict(save_name):
    with open(save_name, 'rb') as f:
        loaded = pickle.load(f)
    return loaded

In [ ]:
senses = ['visual', 'auditory', 'olfactory', 'gustatory', 'tactile', 'proprio']
num_senses = len(senses)

In [8]:
sense_dict = {senses[i]: i for i in range(num_senses)}

BigToM first

In [9]:
import pandas as pd

new_bigtom = pd.read_csv("bigtom-modified.csv", delimiter = ';')
# new_bigtom
# we can edit on top of this

In [10]:
new_bigtom.iloc[0] #['Story - Perception cues']

Story - Context                                 Noor is working as a barista at a busy coffee ...
Story - Desire                                   Noor wants to make a delicious cappuccino for...
Story - Perception cues                          Noor grabs a milk pitcher and fills it with o...
Story - Belief                                   Noor believes that the milk pitcher contains ...
Story - An external event                        A coworker, who didn't hear the customer's re...
Percepts - Aware of event                               Noor sees her coworker swapping the milk.
Percepts - Not aware of event                   Noor does not see her coworker swapping the milk.
Actions - Action given new state                Noor opens the fridge again and reaches for th...
Actions - Action given initial state            Noor makes the cappuccino using the milk in th...
Questions - Belief Question                     Does Noor believe the milk pitcher contains oa...
Questions - Desire Q

In [62]:
import pandas as pd
# !pip install huggingface_hub

splits = {'Long': 'opentom.json', 'ExtraLong': 'opentom_long.json'}
new_opentom = pd.read_json("hf://datasets/SeacowX/OpenToM/" + splits["Long"])
# for i in range(13708):
#     # if df.iloc[i]['question']['type'] == 'attitude':
#     print(new_opentom.iloc[i]['preferences']['observer_mover'])

In [68]:
final_opentom[0]

{'ID': '0-0-0',
 'category': 'pos/hard_neg',
 'sense': 'visual',
 'story': 'Amir likes scarf. Diego hates scarf. Diego entered the patio. Amir entered the patio. Both Diego and Amir noticed that the scarf is in the basket in the patio. In that moment, without exchanging any words, Diego swiftly reached into the basket and snatched the scarf. Amir watched curiously as Diego took a few steps towards a nearby donation bin. With a resolute expression, Diego dropped the scarf into the bin, relieving himself of its presence. And just like that, the scarf that once rested in the patio basket had found a new temporary home in the donation bin, waiting to be discovered by someone who would appreciate its warmth and beauty. Diego turned around to leave the patio, completely unaware that his actions had not gone unnoticed by Amir.',
 'protag_name': 'Amir',
 'percept_cue': 'Amir noticed Diego using only his fingertips to handle the scarf, as if avoiding full contact with it, and then quickly wipin

In [ ]:
story_index = -1
ind_list = {}
for instance in final_opentom:
    st_ind = int(instance['ID'].split('-')[0])
    if story_index == st_ind:
        continue
    story_index = st_ind
    print(story_index)
    ind_list[story_index] = []
    matching = instance['story'].split('. ')[2]
    matching_2 = instance['story'].split('. ')[3]
    for i in range(13708):
        #if new_opentom.iloc[i]['question']['type'] == 'attitude': <-- we can do this in the next step
        if (new_opentom.iloc[i]['plot'].split('.\n')[0] == matching) and (new_opentom.iloc[i]['plot'].split('.\n')[1] == matching_2):
            ind_list[story_index].append(i)
    if story_index == 120:
        break
        
        

In [138]:
# new_opentom.iloc[602]
opentom_instances = {}
for idx in ind_list.keys():
    opentom_instances[idx] = []
    for i in ind_list[idx]:
        if new_opentom.iloc[i]['question']['type'] == 'attitude':
            opentom_instances[idx].append(new_opentom.iloc[i])
        #print(new_opentom.iloc[i]['question']['type'])

In [143]:
for idx in range(121):
    if (len(opentom_instances[idx]) >= 2):
        print(idx)

In [ ]:
# opentom_instances[26]

[]

In [151]:
unique_opentom_labels = {}
# format: {0: [no_belief_change_label, have_belief_change_label]}

In [209]:
# for i in range(15):
#     print(unique_opentom_labels[i])
#     print(opentom_instances[i][0]['preferences'])
#     print(opentom_instances[i][0]['question'])

Here, we use a rule-based method to label the OpenToM dataset after introducing "belief change".

In [232]:
# ind_list
for idx in opentom_instances.keys():
    for question in opentom_instances[idx]:
        unique_opentom_labels[idx] = []
        default_answer = question['question']['answer']
        unique_opentom_labels[idx].append(default_answer)
        
        observer_preference = question['preferences']['observer'].split(' ')[-2]
        observer_belief_of_mover_preference = ""
        if question['preferences']['observer_mover'] == '':
            observer_belief_of_mover_preference = question['preferences']['mover'].split(' ')[-2]
        else:
            observer_belief_of_mover_preference = question['preferences']['observer_mover'].split(' ')[-2]
        
        # use rule-based method to define observer belief of mover preference when there's belief change:
        new_answer = ""
        if (observer_preference == observer_belief_of_mover_preference):
            # after belief change, observer will think they disagree with mover.
            if default_answer == "positive":
                new_answer = 'neutral'
            elif default_answer == 'neutral':
                new_answer = 'negative'
            elif default_answer == 'negative':
                new_answer = 'negative'
            else:
                print('Error')
                print(default_answer)
        else:
            if default_answer == 'positive':
                new_answer = 'positive'
            elif default_answer == 'neutral':
                new_answer = 'positive'
            elif default_answer == 'negative':
                new_answer = 'neutral'
            else:
                print('Error')
                print(default_answer)
        unique_opentom_labels[idx].append(new_answer)
        
        

In [ ]:
unique_opentom_labels

In [234]:
# # print(opentom_instances[7])
# print(opentom_instances[0][0]['narrative'])
# print(opentom_instances[0][0]['preferences'])#['observer_mover'].split(' ')[-2])
# print(opentom_instances[0][0]['preferences']['observer'].split(' ')[-2] == opentom_instances[0][0]['preferences']['observer_mover'].split(' ')[-2])
# print(opentom_instances[0][0]['question'])

In [235]:
import json

def save_jsonl(file_name, path):
    with open(path, "w") as file:
        for item in file_name:
            json_string = json.dumps(item)
            file.write(json_string + '\n')


In [236]:
opentom_system_prompt = "Read and comprehend the following short story. Then, answer the question that follows. "

In [12]:
system_prompt = "Answer the questions based on the context. Keep your answer concise, few words are "
system_prompt += "enough, maximum one sentence. Answer as ’Answer:<option>)<answer>’ \n\n"

In [ ]:
bigtom_control_ToM_ans = [{'ID': idx,
                            'answer_order': [[0,1], [1,0]], # 0 = belief not aware, 1 = belief aware. 
                            'llm_answer': {'belief': ['X','X'], 'action': ['X','X']},
                            'true_answer': {'belief': [0,1], 'action': [0,1]},
                            # 'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                            'reasoning_content': {'belief': ['X','X'], 'action': ['X','X']}} 
                            for idx in range(100)]

bigtom_BC_ToM_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': {'belief': ['X','X'], 'action': ['X','X']},
                    #  'llm_answer': ['X','X','X','X'],
                     'true_answer': {'belief': [instance['belief_change_label'], 1 - instance['belief_change_label']], 'action': [instance['belief_change_label'], 1 - instance['belief_change_label']]},
                    #  'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': {'belief': ['X','X'], 'action': ['X','X']}} 
                    for instance in final_bigtom]

In [237]:
opentom_control_ToM_ans = [{'ID': idx,
                            #'answer_order': [[0,1], [1,0]], # 0 = belief not aware, 1 = belief aware. 
                            'llm_answer': "X",
                            'true_answer': unique_opentom_labels[idx][0],
                            # 'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                            'reasoning_content': "X"} 
                            for idx in unique_opentom_labels.keys()]

opentom_BC_ToM_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     #'answer_order': [[0,1], [1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': "X",
                    #  'llm_answer': ['X','X','X','X'],
                     'true_answer': unique_opentom_labels[int(instance['ID'].split('-')[0])][instance['belief_change_label']], #{'belief': [instance['belief_change_label'], 1 - instance['belief_change_label']], 'action': [instance['belief_change_label'], 1 - instance['belief_change_label']]},
                    #  'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': "X"} 
                    for instance in final_opentom if int(instance['ID'].split('-')[0]) in unique_opentom_labels.keys()] #aloha

In [240]:
# len(final_opentom)

In [241]:
model = "gpt-4o"

In [ ]:

# Noor is working as a barista at a busy coffee shop.  - context
# Noor wants to make a delicious cappuccino for a customer who asked for oat milk.   desire
# Noor grabs a milk pitcher and fills it with oat milk.  - action.
model = 'gpt-4o'
# # positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# First 
for story_id in range(100): # set to 100 later
    print(f'{story_id}-th story')
    for mode in ['belief', 'action']: # forward belief, forward action
        for k in range(2):
            control = f"Story: {new_bigtom.iloc[story_id]['Story - Context']} {new_bigtom.iloc[story_id]['Story - Desire']} "# {final_bigtom[story_id]['event']} " 
            control += f"{new_bigtom.iloc[story_id]['Story - Perception cues']} "
            # # Event:
            # control += f"{new_bigtom.iloc[story_id]['Random Event']} "
            # control += f"{new_bigtom.iloc[story_id]['Percepts of R.E. - not Aware of random event']} "
            if mode == 'belief':
                control += f"\n Question: {new_bigtom.iloc[story_id]['Questions - Belief Question']} \n"
                if k == 0:
                    control += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Belief not Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Belief Aware']} \n"
                else:
                    control += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Belief Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Belief not Aware']} \n"
            else:
                control += f"\n Question: {new_bigtom.iloc[story_id]['Questions - Action Question']} \n"
                if k == 0:
                    control += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Action not Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Action Aware']} \n"
                else:
                    control += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Action Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Action not Aware']} \n"
                    
            prompt = system_prompt + control
            # print(prompt)
            
            # # Add back later:
            # prompt += "\nProvide the answer as a single letter:"

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
            output = output[0]#.split(' ')[1]
            llm_answer = output#[0]

            bigtom_control_ToM_ans[story_id]['llm_answer'][mode][k] = llm_answer # we can do post-processing later. 

                    
            output_tokens += count_tokens(output[0])
            # save_jsonl(bigtom_control_ToM_ans, f'bigtom_control_ToM_ans_{model}_simple.jsonl')
            save_jsonl(bigtom_control_ToM_ans, f'bigtom_task0_before_belief_change_{model}.jsonl')
            
    if story_id % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)


# bigtom_task1_llm_answer = [instance['llm_answer'] for instance in bigtom_task1_ans]
# bigtom_task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in bigtom_task1_ans]
# bigtom_task1_llm_answer = np.array(bigtom_task1_llm_answer)
# bigtom_task1_true_answer = np.array(bigtom_task1_true_answer)
# bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)

# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# bigtom_task1_total = np.zeros((100, 3, 8))
# for i, instance in enumerate(bigtom_task1_ans):
#     ider = instance['ID']
#     story_idx = int(ider.split('-')[0])
#     gt_idx = gt_id[instance['groundtruth_classification']]
#     sense_idx = sense_dict[instance['sense']]
#     bigtom_task1_total[story_idx][gt_idx][sense_idx] = 1
#     bigtom_task1_correct[story_idx][gt_idx][sense_idx] = bigtom_task1_accuracy[i]
# eps = 1e-10
# print("BigToM Task 1 TPR, TNR-hard, TNR-easy: ", bigtom_task1_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task1_total.sum(axis = 0).sum(axis = 1) + eps))


0-th story
Time taken for these 5 stories: 1.8966450691223145 seconds.
1-th story
2-th story
3-th story
4-th story
5-th story
Time taken for these 5 stories: 9.920233011245728 seconds.
6-th story
7-th story
8-th story
9-th story
10-th story
Time taken for these 5 stories: 9.956781148910522 seconds.
11-th story
12-th story
13-th story
14-th story
15-th story
Time taken for these 5 stories: 40.71083188056946 seconds.
16-th story
17-th story
18-th story
19-th story
20-th story
Time taken for these 5 stories: 10.356099128723145 seconds.
21-th story
22-th story
23-th story
24-th story
25-th story
Time taken for these 5 stories: 9.610530138015747 seconds.
26-th story
27-th story
28-th story
29-th story
30-th story
Time taken for these 5 stories: 40.86195468902588 seconds.
31-th story
32-th story
33-th story
34-th story
35-th story
Time taken for these 5 stories: 10.241355895996094 seconds.
36-th story
37-th story
38-th story
39-th story
40-th story
Time taken for these 5 stories: 10.61562490

In [ ]:
# See result
# dataset = 'opentom'
# task = 'task3'
# model = 'deepseek-reasoner'
# with open(f'{dataset}_{task}_ans_{model}.jsonl') as f: # put story here
#         task_ans = np.array([json.loads(line) for line in f])
# print(len(task_ans))
# with open(f'{dataset}_task1_ans_{model}.jsonl') as f: # put story here
#         IDs = np.array([json.loads(line)['ID'] for line in f])
# filter = ''
model = 'gpt-35-turbo'

with open(f'bigtom_task0_before_belief_change_{model}.jsonl') as f:
    task_ans = np.array([json.loads(line) for line in f])#[:20]
# task_ans = task_ans[:10] # remove later

belief_llm_answer = [instance['llm_answer']['belief'] for instance in task_ans]
belief_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer']['belief'])] for instance in task_ans]
action_llm_answer = [instance['llm_answer']['action'] for instance in task_ans]
action_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer']['action'])] for instance in task_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
belief_llm_answer = np.array(belief_llm_answer)
belief_true_answer = np.array(belief_true_answer)
action_llm_answer = np.array(action_llm_answer)
action_true_answer = np.array(action_true_answer)


# I think we can just use accuracy for this one.
print('Forward Belief Accuracy:', (belief_llm_answer == belief_true_answer).mean())
print('Forward Action Accuracy:', (action_llm_answer == action_true_answer).mean())


Forward Belief Accuracy: 0.97
Forward Action Accuracy: 0.825


In [127]:
# belief_llm_answer

Actual ToM Test

In [129]:
# final_bigtom

In [ ]:
# First, do the control experiments. 
# Noor is working as a barista at a busy coffee shop.  - context
# Noor wants to make a delicious cappuccino for a customer who asked for oat milk.   desire
# Noor grabs a milk pitcher and fills it with oat milk.  - action.
model = 'gpt-35-turbo'
# # positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# First 
for i, instance in enumerate(final_bigtom): # set to all remaining later. 
    print(f'{i}-th instance')
    for mode in ['belief', 'action']: # forward belief, forward action
        for k in range(2):
            story_id = final_bigtom[i]['ID'].split('-')[0]
            story_id = int(story_id)
            after_change = f"Story: {new_bigtom.iloc[story_id]['Story - Context']} {new_bigtom.iloc[story_id]['Story - Desire']} {new_bigtom.iloc[story_id]['Story - Context']} "# {final_bigtom[story_id]['event']} " 
            after_change += f"{new_bigtom.iloc[story_id]['Story - Perception cues']} "
            # Event:
            after_change += f"{new_bigtom.iloc[story_id]['Story - An external event']} "
            # after_change += f"Initially, {new_bigtom.iloc[story_id]['Percepts - Not aware of event']} "
            after_change += f"Regarding the event, the main character {final_bigtom[i]['protag_name']} only encounters the following percept: {final_bigtom[i]['percept_cue']} "
            if mode == 'belief':
                after_change += f"\n Question: {new_bigtom.iloc[story_id]['Questions - Belief Question']} \n"
                if k == 0:
                    after_change += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Belief not Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Belief Aware']} \n"
                else:
                    after_change += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Belief Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Belief not Aware']} \n"
            else:
                after_change += f"\n Question: {new_bigtom.iloc[story_id]['Questions - Action Question']} \n"
                if k == 0:
                    after_change += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Action not Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Action Aware']} \n"
                else:
                    after_change += f"Choose one of the following: \n A){new_bigtom.iloc[story_id]['Answers - Action Aware']} \n B){new_bigtom.iloc[story_id]['Answers - Action not Aware']} \n"
                    
            prompt = system_prompt + after_change
            # print(prompt)
            
            # # Add back later:
            # prompt += "\nProvide the answer as a single letter:"

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
            output = output[0]#.split(' ')[1]
            llm_answer = output#[0]

            bigtom_BC_ToM_ans[i]['llm_answer'][mode][k] = llm_answer # we can do post-processing later. 

                    
            output_tokens += count_tokens(output[0])
            #save_jsonl(bigtom_BC_ToM_ans, f'bigtom_BC_ToM_ans_{model}_without_initial.jsonl')
            save_jsonl(bigtom_BC_ToM_ans, f'bigtom_task0_after_belief_change_ans_{model}.jsonl')
            
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)


# bigtom_task1_llm_answer = [instance['llm_answer'] for instance in bigtom_task1_ans]
# bigtom_task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in bigtom_task1_ans]
# bigtom_task1_llm_answer = np.array(bigtom_task1_llm_answer)
# bigtom_task1_true_answer = np.array(bigtom_task1_true_answer)
# bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)

# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# bigtom_task1_total = np.zeros((100, 3, 8))
# for i, instance in enumerate(bigtom_task1_ans):
#     ider = instance['ID']
#     story_idx = int(ider.split('-')[0])
#     gt_idx = gt_id[instance['groundtruth_classification']]
#     sense_idx = sense_dict[instance['sense']]
#     bigtom_task1_total[story_idx][gt_idx][sense_idx] = 1
#     bigtom_task1_correct[story_idx][gt_idx][sense_idx] = bigtom_task1_accuracy[i]
# eps = 1e-10
# print("BigToM Task 1 TPR, TNR-hard, TNR-easy: ", bigtom_task1_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task1_total.sum(axis = 0).sum(axis = 1) + eps))


0-th instance
Time taken for these 5 stories: 2.4821488857269287 seconds.
1-th instance
2-th instance
3-th instance
4-th instance
5-th instance
Time taken for these 5 stories: 10.120771169662476 seconds.
6-th instance
7-th instance
8-th instance
9-th instance
10-th instance
Time taken for these 5 stories: 10.181493043899536 seconds.
11-th instance
12-th instance
13-th instance
14-th instance
15-th instance
Time taken for these 5 stories: 40.27358388900757 seconds.
16-th instance
17-th instance
18-th instance
19-th instance
20-th instance
Time taken for these 5 stories: 9.934164047241211 seconds.
21-th instance
22-th instance
23-th instance
24-th instance
25-th instance
Time taken for these 5 stories: 10.783399105072021 seconds.
26-th instance
27-th instance
28-th instance
29-th instance
30-th instance
Time taken for these 5 stories: 39.99672174453735 seconds.
31-th instance
32-th instance
33-th instance
34-th instance
35-th instance
Time taken for these 5 stories: 11.133774995803833 se

In [25]:
len(bigtom_BC_ToM_ans)

1169

In [43]:
# Post processing:
# postprocessing:
# bigtom_control_ToM_ans[story_id]['llm_answer'][mode][k]
for i in range(len(bigtom_BC_ToM_ans)):
    for mode in ['belief','action']:
        for k in range(2):
            letter = bigtom_BC_ToM_ans[i]['llm_answer'][mode][k].split(':')[1].lstrip()[0]
            # if len(letter) == 1:
            #     # print(letter)
            # else:
            # letter = letter[1].lstrip()[0]
            if (letter not in ['A','B']):
                print(letter)
            bigtom_BC_ToM_ans[i]['llm_answer'][mode][k] = letter


In [ ]:
# for i in range(len(bigtom_BC_ToM_ans)):
#     for mode in ['belief','action']:
#         for k in range(2):
#             letter = bigtom_BC_ToM_ans[i]['llm_answer'][mode][k]
#             if letter not in ['A', 'B']:
#                 print(letter)

In [45]:
# bigtom_BC_ToM_ans

In [ ]:
save_jsonl(bigtom_BC_ToM_ans, f'bigtom_task0_after_belief_change_ans_{model}.jsonl')

In [ ]:
# See result
# dataset = 'opentom'
# task = 'task3'
# model = 'deepseek-reasoner'
# with open(f'{dataset}_{task}_ans_{model}.jsonl') as f: # put story here
#         task_ans = np.array([json.loads(line) for line in f])
# print(len(task_ans))
# with open(f'{dataset}_task1_ans_{model}.jsonl') as f: # put story here
#         IDs = np.array([json.loads(line)['ID'] for line in f])
# filter = ''
from sklearn.metrics import classification_report
model = 'gpt-35-turbo'

with open(f'bigtom_task0_after_belief_change_ans_{model}.jsonl') as f:
    task_ans = np.array([json.loads(line) for line in f])#[:100]
# task_ans = task_ans[:10] # remove later

belief_llm_answer = [instance['llm_answer']['belief'] for instance in task_ans]
belief_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer']['belief'])] for instance in task_ans]
action_llm_answer = [instance['llm_answer']['action'] for instance in task_ans]
action_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer']['action'])] for instance in task_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
belief_llm_answer = np.array(belief_llm_answer)
belief_true_answer = np.array(belief_true_answer)
action_llm_answer = np.array(action_llm_answer)
action_true_answer = np.array(action_true_answer)

def print_values(d):
    """Recursively print only the values in a nested dictionary."""
    if isinstance(d, dict):
        for val in d.values():
            print_values(val)
    else:
        print(f"{d:.3f}", end = ' ')


for mode in ['belief', 'action']:
    print(mode, 'mode\n\n')
    converted_llm_answer_hard = []
    converted_true_answer_hard = []
    converted_llm_answer_total = []
    converted_true_answer_total = []
    converted_llm_answer_easy_only = []
    converted_true_answer_easy_only = []
    for idx, instance in enumerate(task_ans):
        if instance['groundtruth_classification'] == 'positive':
            for i in range(len(instance['llm_answer'][mode])):
                # insert to hard array
                # llm_answer = instance['llm_answer'][i]
                llm_correct = instance['llm_answer'][mode][i] == np.array(['A', 'B'])[np.array(instance['true_answer'][mode])][i]


                converted_true_answer_hard.append(1)
                converted_llm_answer_hard.append(int(llm_correct))
                converted_true_answer_total.append(1)
                converted_llm_answer_total.append(int(llm_correct))
                converted_true_answer_easy_only.append(1)
                converted_llm_answer_easy_only.append(int(llm_correct))
        else: # negative
            for i in range(len(instance['llm_answer'][mode])):
                llm_correct = instance['llm_answer'][mode][i] == np.array(['A', 'B'])[np.array(instance['true_answer'][mode])][i]

                converted_true_answer_total.append(0)
                converted_llm_answer_total.append(1-int(llm_correct))
                if instance['groundtruth_classification'] == 'hard_negative':
                    converted_true_answer_hard.append(0)
                    converted_llm_answer_hard.append(1-int(llm_correct))
                else:
                    converted_true_answer_easy_only.append(0)
                    converted_llm_answer_easy_only.append(1-int(llm_correct))


    hard_neg_only_report = classification_report(converted_true_answer_hard,converted_llm_answer_hard, digits = 3, output_dict = True)
    easy_neg_only_report = classification_report(converted_true_answer_easy_only,converted_llm_answer_easy_only, digits = 3, output_dict = True)
    all_neg_report = classification_report(converted_true_answer_total,converted_llm_answer_total, digits = 3, output_dict = True)
    return_dict = {'Class-Specific': {'Positive': {k: v for k,v in all_neg_report['1'].items() if k != 'support'}, 'Hard_Negative': {k: v for k,v in hard_neg_only_report['0'].items() if k != 'support'}, 'Easy_Negative': {k: v for k,v in easy_neg_only_report['0'].items() if k != 'support'}},
                'Combined': {'Positive_and_Hard_Neg_only':{'accuracy':hard_neg_only_report['accuracy'] if hard_neg_only_report['accuracy'] is not None else hard_neg_only_report['micro avg']['f1-score'], 'macro-f1' : hard_neg_only_report['macro avg']['f1-score']},
                                'Total':{'accuracy':all_neg_report['accuracy'] if all_neg_report['accuracy'] is not None else all_neg_report['micro avg']['f1-score'], 'macro-f1' : all_neg_report['macro avg']['f1-score']}}}
    print_values(return_dict)
    


# # I think we can just use accuracy for this one.
# print('Forward Belief Accuracy:', (belief_llm_answer == belief_true_answer).mean())
# print('Forward Action Accuracy:', (action_llm_answer == action_true_answer).mean())


belief mode


0.465 0.844 0.599 0.393 0.205 0.270 0.625 0.308 0.413 0.634 0.513 0.516 0.495 action mode


0.434 0.856 0.576 0.294 0.122 0.172 0.523 0.187 0.276 0.614 0.461 0.460 0.416 

In [142]:
# correct:
bigtom_BC_ToM_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': {'belief': ['X','X'], 'action': ['X','X']},
                    #  'llm_answer': ['X','X','X','X'],
                     'true_answer': {'belief': [instance['belief_change_label'], 1 - instance['belief_change_label']], 'action': [instance['belief_change_label'], 1 - instance['belief_change_label']]},
                    #  'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']],
                     'reasoning_content': {'belief': ['X','X'], 'action': ['X','X']}} 
                    for instance in final_bigtom]

for i in range(100):
    bigtom_BC_ToM_ans[i]['llm_answer'] = task_ans[i]['llm_answer']


In [139]:
bigtom_BC_ToM_ans

[{'ID': '0-0-0',
  'groundtruth_classification': 'positive',
  'sense': 'visual',
  'answer_order': [[0, 1], [1, 0]],
  'llm_answer': {'belief': ['B', 'B'], 'action': ['B', 'A']},
  'true_answer': {'belief': [1, 0], 'action': [1, 0]},
  'reasoning_content': {'belief': ['X', 'X'], 'action': ['X', 'X']}},
 {'ID': '0-0-1',
  'groundtruth_classification': 'hard_negative',
  'sense': 'visual',
  'answer_order': [[0, 1], [1, 0]],
  'llm_answer': {'belief': ['B', 'A'], 'action': ['B', 'A']},
  'true_answer': {'belief': [0, 1], 'action': [0, 1]},
  'reasoning_content': {'belief': ['X', 'X'], 'action': ['X', 'X']}},
 {'ID': '0-0-2',
  'groundtruth_classification': 'positive',
  'sense': 'visual',
  'answer_order': [[0, 1], [1, 0]],
  'llm_answer': {'belief': ['A', 'B'], 'action': ['A', 'B']},
  'true_answer': {'belief': [1, 0], 'action': [1, 0]},
  'reasoning_content': {'belief': ['X', 'X'], 'action': ['X', 'X']}},
 {'ID': '0-0-3',
  'groundtruth_classification': 'positive',
  'sense': 'auditor

In [ ]:



# prompt format:
i = 0
story_id = final_bigtom[i]['ID'].split('-')[0]
story_id = int(story_id)
after_change = control = f"Story: {final_bigtom[i]['story']} {final_bigtom[i]['event']} " 
control += f"{new_bigtom.iloc[story_id]['Percepts - Not aware of event ']} "
after_change += f"Initially, {new_bigtom.iloc[story_id]['Percepts - Not aware of event ']} "
after_change += f"Subsequently, {final_bigtom[i]['percept_cue']} "


after_change_forward_belief = after_change + f"\n Question: {new_bigtom.iloc[story_id]['Questions - Belief Question']} \n"
control_forward_belief = control + f"\n Question: {new_bigtom.iloc[story_id]['Questions - Belief Question']} \n"
after_change_forward_action = after_change + f"\n Question: {new_bigtom.iloc[story_id]['Questions - Action Question']} \n"
control_forward_action = control + f"\n Question: {new_bigtom.iloc[story_id]['Questions - Action Question']} \n"

after_change_forward_belief += f"Choose one of the following: \n a){final_bigtom[i]['answer_options'][1]} \n b){final_bigtom[i]['answer_options'][0]} \n"
control_forward_belief += f"Choose one of the following: \n a){final_bigtom[i]['answer_options'][1]} \n b){final_bigtom[i]['answer_options'][0]} \n"
after_change_forward_action += f"Choose one of the following: \n a){new_bigtom.iloc[story_id]['Answers - Action not Aware']} \n b){new_bigtom.iloc[story_id]['Answers - Action Aware']} \n"
control_forward_action += f"Choose one of the following: \n a){new_bigtom.iloc[story_id]['Answers - Action not Aware']} \n b){new_bigtom.iloc[story_id]['Answers - Action Aware']} \n"



In [ ]:
opentom_system_prompt += "\n\n"

In [ ]:
model = 'gpt-4o'
# # positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# First 
for idx, story_id in enumerate(list(unique_opentom_labels.keys())): 
    # if idx not in [33, 49, 101]:
    #     continue
    print(f'{story_id}-th story')
    control = f"Story: {opentom_instances[story_id][0]['narrative']} \n\n" #{new_bigtom.iloc[story_id]['Story - Desire']} "# {final_bigtom[story_id]['event']} " 
    control += f"{opentom_instances[story_id][0]['question']['question']} \n\n"
    control += 'Answer with "positive", "neutral", or "negative". '
    control += 'Answer the question without any explanation.'
            
    prompt = opentom_system_prompt + control
    
    # # Add back later:
    # prompt += "\nProvide the answer as a single letter:"

    input_tokens += count_tokens(prompt)
    
    # print(prompt)
    # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
    
    
    # Add back later
    output = LM_generation(prompt, temperature = 0, max_tokens = 5, model = model, system_msg = 'You are an expert in modeling other\'s mental state.')
    output = output[0]#.split(' ')[1]
    llm_answer = output#[0]
    #print(llm_answer)

    opentom_control_ToM_ans[idx]['llm_answer'] = llm_answer # we can do post-processing later. 

            
    output_tokens += count_tokens(output[0])
    #save_jsonl(opentom_control_ToM_ans, f'opentom_control_ToM_ans_{model}_simple.jsonl')
    save_jsonl(opentom_control_ToM_ans, f'opentom_task0_before_belief_change_{model}.jsonl')
            
    if story_id % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

In [ ]:
for i, idx in enumerate(unique_opentom_labels.keys()):
    opentom_control_ToM_ans[i]['llm_answer'] = opentom_control_ToM_ans[i]['llm_answer'].split(': ')[-1]
    opentom_control_ToM_ans[i]['llm_answer'] = opentom_control_ToM_ans[i]['llm_answer'].strip('<>')
    print(i, opentom_control_ToM_ans[i]['llm_answer'])

In [ ]:
#save_jsonl(opentom_control_ToM_ans, f'opentom_control_ToM_ans_{model}_simple.jsonl')
save_jsonl(opentom_control_ToM_ans, f'opentom_task0_before_belief_change_{model}.jsonl')

In [ ]:
from sklearn.metrics import classification_report
llm_answer = [instance['llm_answer'] for instance in opentom_control_ToM_ans]
true_answer = [instance['true_answer'] for instance in opentom_control_ToM_ans]
print(classification_report(true_answer, llm_answer))

In [ ]:
model = 'gpt-4o'
# # positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# First 
idx = 0
for instance in final_opentom[:4]: 
    # if idx not in [33, 49, 101]: #86-0-7 
# 90-0-4 
# 9-1-6 
# 9-1-1 
    #     continue
    
    story_id = int(instance['ID'].split('-')[0])
    # if instance['ID'] not in ['86-0-7', '90-0-4', '9-1-6','9-1-1']:
    #     continue
    print(f'{idx}-th instance')
    control = f"Story: {opentom_instances[story_id][0]['narrative']} \n\n" #{new_bigtom.iloc[story_id]['Story - Desire']} "# {final_bigtom[story_id]['event']} " 
    
    control += f"After the story, the main character {instance['protag_name']} only encounters the following percept: {instance['percept_cue']} \n\n"
    control += f"{opentom_instances[story_id][0]['question']['question']} \n"
    control += 'Answer with "positive", "neutral", or "negative". '
    control += 'Answer the question without any explanation.'
    
            
    prompt = opentom_system_prompt + control
    # print(prompt)
    
    # # Add back later:
    # prompt += "\nProvide the answer as a single letter:"

    input_tokens += count_tokens(prompt)
    
    # print(prompt)
    # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
    
    
    # Add back later
    output = LM_generation(prompt, temperature = 0, max_tokens = 8, model = model, system_msg = 'You are an expert in modeling other\'s mental state.')
    output = output[0]#.split(' ')[1]
    llm_answer = output#[0]
    # print(llm_answer)

    opentom_BC_ToM_ans[idx]['llm_answer'] = llm_answer # we can do post-processing later. 

            
    output_tokens += count_tokens(output[0])
    #save_jsonl(opentom_BC_ToM_ans, f'opentom_BC_ToM_ans_{model}_without_initial.jsonl')
    save_jsonl(opentom_BC_ToM_ans, f'opentom_task0_after_belief_change_{model}.jsonl')
            
    if story_id % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()
    idx += 1


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

In [ ]:
# save_jsonl(opentom_BC_ToM_ans, f'opentom_BC_ToM_ans_{model}_without_initial.jsonl')
save_jsonl(opentom_BC_ToM_ans, f'opentom_task0_after_belief_change_{model}.jsonl')

In [ ]:
from sklearn.metrics import classification_report
llm_answer = [instance['llm_answer'] for instance in opentom_BC_ToM_ans]
true_answer = [instance['true_answer'] for instance in opentom_BC_ToM_ans]
print(classification_report(true_answer, llm_answer))

In [ ]:
# By Case

llm_answer = [instance['llm_answer'] for instance in opentom_BC_ToM_ans if instance['groundtruth_classification'] != 'easy_negative']
true_answer = [instance['true_answer'] for instance in opentom_BC_ToM_ans if instance['groundtruth_classification'] != 'easy_negative']
print(classification_report(true_answer, llm_answer))